# Travel Assistant + Tavily + Weather API tools
## Please note that this notebook uses the latest way to create and use ReAct agents and OpenAI models using Langchain

### PLease visit https://docs.langchain.com/oss/python/langchain/agents and https://docs.langchain.com/oss/python/integrations/chat/openai for more details

In [1]:
import os
import requests
from typing import Dict, Any
from bs4 import BeautifulSoup
from langchain.tools import tool
from langchain.agents import create_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain_core.messages import AIMessage
from langchain_core.messages import HumanMessage, SystemMessage


In [2]:
tavily_api_key = "<API KEY>"
weather_api_key = "<API KEY>"
openai_api_key = "<API KEY>"

os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["TAVILY_API_KEY"] = tavily_api_key
os.environ["WEATHERAPI_KEY"] = weather_api_key

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
WEATHERAPI_KEY = os.getenv("WEATHERAPI_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

HEADERS = {"User-Agent": "TravelAssistant/1.0"}

In [3]:
def extract_text(html: str) -> str:
    soup = BeautifulSoup(html, "html.parser")
    for tag in soup(["script", "style"]):
        tag.decompose()

    text = soup.get_text(" ", strip=True)
    return text[:60000]

In [4]:
@tool
def weather_tool(city: str) -> str:
    """Get the current weather for the given city using WeatherAPI."""
    try:
        resp = requests.get(
            "http://api.weatherapi.com/v1/current.json",
            params={"key": WEATHERAPI_KEY, "q": city},
            timeout=10,
        )
        resp.raise_for_status()
    except Exception as e:
        return f"Weather API error: {e}"

    data = resp.json()
    loc = data["location"]
    cur = data["current"]

    return (
        f"Weather in {loc['name']}, {loc['country']} ({loc['localtime']}):\n"
        f"- Condition: {cur['condition']['text']}\n"
        f"- Temp: {cur['temp_c']}°C (Feels {cur['feelslike_c']}°C)\n"
        f"- Humidity: {cur['humidity']}%\n"
        f"- Wind: {cur['wind_kph']} km/h {cur['wind_dir']}\n"
    )


In [5]:
@tool
def tavily_tourist_search(city: str) -> str:
    """Search tourist attractions for the given city using Tavily and summarize webpage contents."""

    search = TavilySearchResults(k=5, api_key=TAVILY_API_KEY)
    query = f"Top tourist attractions in {city} travel guide blog"
    results = search.run(query)

    urls = [i["url"] for i in results if "url" in i][:5]

    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, api_key = os.getenv("OPENAI_API_KEY"))

    summaries = []

    for url in urls:
        try:
            r = requests.get(url, headers=HEADERS, timeout=8)
            r.raise_for_status()

            text = extract_text(r.text)
            text = text[:12000] 
            
            prompt = [
                {"role": "system",
                    "content": "You are a travel content summarizer. Produce a crisp summary."},
                {"role": "user",
                    "content": f"Summarize the following webpage content:\n\n{text}"}]

            response = llm.invoke(prompt)
            summary = response.content

            summaries.append(f"URL: {url}\nSummary: {summary}\n")

        except Exception as e:
            summaries.append(f"URL: {url}\nError summarizing page: {e}\n")

    return "\n".join(summaries)

In [6]:


def build_agent():
    model = ChatOpenAI(
        model="gpt-3.5-turbo",
        temperature=0,
        api_key = os.getenv("OPENAI_API_KEY")
    )

    tools = [weather_tool, tavily_tourist_search]
    agent = create_agent(
        model=model,
        tools=tools,
        system_prompt="You are an AI travel research assistant. Use your tools wisely to answer the user."
    )

    return agent

In [40]:
def summarizer_node(state: Dict[str, Any]) -> Dict[str, Any]:
    """Summarizes weather + tourist tool output into an executive travel report."""
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, api_key = os.getenv("OPENAI_API_KEY"))
    weather = state["weather"]
    tourist = state["tourist"]
    city = state["city"]

    prompt = [
        {"role": "system",
            "content": "You are an expert travel report writer. Produce executive-level summaries."},
        {"role": "user",
            "content": f"""
    Write an executive blog-style travel report for **{city}**.
    
    Use the **current weather conditions** below to enrich the summary:
    
    === CURRENT WEATHER ===
    {weather}
    
    === TOURIST INFORMATION ===
    {tourist}
    
    Make sure the final report explicitly includes:
    - Current weather condition (sunny, cloudy, rainy, etc.)
    - Temperature and how it feels
    - Humidity levels
    - Wind conditions
    - How the weather affects travel experience
    - Whether the weather is ideal for visiting right now
    
    Output format:
    **Executive Summary**
    **Current Weather Conditions**
    **Key Insights**
    **Best Time to Visit**
    **Top Attractions**
    **Final Recommendation**
    """
        }]

    response = llm.invoke(prompt)
    print(response)
    final = response.content

    return {
        "final_report": final
    }

In [32]:
class FlowState(Dict):
    city: str
    weather: str
    tourist: str
    final_report: str


def create_flow():
    agent = build_agent()

    def agent_node(state: FlowState):
        """Runs the Agent and triggers tools."""

        city = state["city"]

        instruction = f"""
Research the city '{city}' using:
1. weather_tool
2. tavily_tourist_search

Return output ONLY in JSON:
{{
  "weather": "...",
  "tourist": "..."
}}
"""

        result = agent.invoke({
            "messages": [
                {"role": "user", "content": instruction}
            ]
        })

        final_ai_messages = [
            msg for msg in result["messages"]
            if isinstance(msg, AIMessage) and msg.content.strip()
        ]

        final_msg = final_ai_messages[-1].content.strip()
        parsed = json.loads(final_msg)

        return {
            "weather": parsed.get("weather", ""),
            "tourist": parsed.get("tourist", "")
        }

    graph = StateGraph(FlowState)
    graph.add_node("agent", agent_node)
    graph.add_node("summarizer", summarizer_node)

    graph.set_entry_point("agent")
    graph.add_edge("agent", "summarizer")
    graph.add_edge("summarizer", END)

    return graph.compile()


In [42]:
flow = create_flow()
result = flow.invoke({"city": "Chennai"})
print(result["final_report"])

**Executive Summary**
Chennai, known for its rich cultural heritage and vibrant atmosphere, offers a blend of historical sites, beautiful beaches, and delectable cuisine. The current weather conditions add a touch of mist to the city's charm, creating a mystical ambiance for travelers to explore.

**Current Weather Conditions**
Weather in Chennai, India (2025-11-18 14:40):
- Condition: Mist
- Temp: 26.2°C (Feels 29.9°C)
- Humidity: 94%
- Wind: 23.4 km/h N

**Key Insights**
The misty weather in Chennai enhances the city's allure, making it an ideal time to immerse oneself in its cultural tapestry. The high humidity levels may create a cozy feel, perfect for leisurely strolls along Marina Beach or exploring the intricate architecture of Kapaleeshwarar Temple.

**Best Time to Visit**
While Chennai is a year-round destination, the period between November and February offers the most pleasant weather for sightseeing and outdoor activities. The current misty conditions provide a unique exper

In [53]:
from IPython.display import display, Markdown
display(Markdown(result["final_report"]))

**Executive Summary**
Chennai, known for its rich cultural heritage and vibrant atmosphere, offers a blend of historical sites, beautiful beaches, and delectable cuisine. The current weather conditions add a touch of mist to the city's charm, creating a mystical ambiance for travelers to explore.

**Current Weather Conditions**
Weather in Chennai, India (2025-11-18 14:40):
- Condition: Mist
- Temp: 26.2°C (Feels 29.9°C)
- Humidity: 94%
- Wind: 23.4 km/h N

**Key Insights**
The misty weather in Chennai enhances the city's allure, making it an ideal time to immerse oneself in its cultural tapestry. The high humidity levels may create a cozy feel, perfect for leisurely strolls along Marina Beach or exploring the intricate architecture of Kapaleeshwarar Temple.

**Best Time to Visit**
While Chennai is a year-round destination, the period between November and February offers the most pleasant weather for sightseeing and outdoor activities. The current misty conditions provide a unique experience, especially for those seeking a more atmospheric visit.

**Top Attractions**
Must-visit attractions in Chennai include Marina Beach, Kapaleeshwarar Temple, and Fort St. George. Visitors can also indulge in the city's art scene, culinary delights, and shopping experiences in bustling areas like T. Nagar. The misty weather adds a mystical touch to these already captivating sites.

**Final Recommendation**
With the current misty weather enveloping Chennai, now is a great time to explore the city's diverse offerings. Whether you're a history buff, a food enthusiast, or a beach lover, Chennai has something for everyone. Embrace the enchanting mist and embark on a memorable journey through this cultural gem of South India.